In [5]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from mtl_pytorch.layer_node import Conv2dNode, BN2dNode

from mtl_pytorch.trainer import Trainer
from data.heads.pixel2pixel import ASPPHeadNode

from data.dataloader.cityscapes_dataloader import CityScapes
from data.metrics.pixel2pixel_loss import CityScapesCriterions
from data.metrics.pixel2pixel_metrics import CityScapesMetrics

from mobilenetv2 import mobilenet_v2

from mtl_model import mtl_model

from data.dataloader.nyuv2_dataloader import NYU_v2
from data.metrics.pixel2pixel_loss import NYUCriterions
from data.metrics.pixel2pixel_metrics import NYUMetrics

from data.dataloader.taskonomy_dataloader import Taskonomy
from data.metrics.pixel2pixel_loss import TaskonomyCriterions
from data.metrics.pixel2pixel_metrics import TaskonomyMetrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

In [6]:
dataroot = 'datasets/Cityscapes/'

headsDict = nn.ModuleDict()
trainDataloaderDict = {}
valDataloaderDict = {}
criterionDict = {}
metricDict = {}

tasks = ['segment_semantic', 'depth_zbuffer']
task_cls_num = {'segment_semantic': 19, 'depth_zbuffer': 1}
for task in tasks:
    headsDict[task] = ASPPHeadNode(1280, task_cls_num[task])

    # For model trainer
    dataset = CityScapes(dataroot, 'train', task, crop_h=224, crop_w=224)
    trainDataloaderDict[task] = DataLoader(dataset, 16, shuffle=True)

    dataset = CityScapes(dataroot, 'test', task)
    valDataloaderDict[task] = DataLoader(dataset, 16, shuffle=True)

    criterionDict[task] = CityScapesCriterions(task)
    metricDict[task] = CityScapesMetrics(task)

# Model

In [7]:
mtlmodel = mobilenet_v2(False, heads_dict=headsDict)

# Train

In [8]:
checkpoint = 'checkpoint/'
trainer = Trainer(mtlmodel, trainDataloaderDict, valDataloaderDict, criterionDict, metricDict, 
                  print_iters=100, val_iters=500, save_num=1, policy_update_iters=100)

### pre_train

In [9]:
trainer.pre_train(iters=1, lr=0.0001, savePath=checkpoint+'Cityscapes/')

100%|██████████| 1/1 [00:44<00:00, 44.65s/it]


### alter_train

In [10]:
loss_lambda = {'segment_semantic': 1, 'depth_zbuffer': 1, 'policy':0.0005}
trainer.alter_train_with_reg(iters=1, policy_network_iters=(100,400), policy_lr=0.01, network_lr=0.0001,
                             loss_lambda=loss_lambda,
                             savePath=checkpoint+'Cityscapes/')

100%|██████████| 1/1 [00:18<00:00, 18.01s/it]


### sample policy from trained policy distribution and save

In [11]:
policy_list = {'segment_semantic': [], 'depth_zbuffer': []}
name_list = {'segment_semantic': [], 'depth_zbuffer': []}

In [12]:
for name, param in mtlmodel.named_parameters():
    if 'policy' in name and not torch.eq(param, torch.tensor([0., 0., 0.])).all():
        if 'segment_semantic' in name:
            policy_list['segment_semantic'].append(param.data.cpu().detach().numpy())
            name_list['segment_semantic'].append(name)
        elif 'depth_zbuffer' in name:
            policy_list['depth_zbuffer'].append(param.data.cpu().detach().numpy())
            name_list['depth_zbuffer'].append(name)

In [13]:
from collections import OrderedDict
from scipy.special import softmax

sample_policy_dict = OrderedDict()
for task in tasks:
    for name, policy in zip(name_list[task], policy_list[task]):
        # distribution = softmax(policy, axis=-1
        distribution = softmax(policy, axis=-1)
        distribution /= sum(distribution)

        choice = np.random.choice((0,1,2), p=distribution)
        if choice == 0:
            sample_policy_dict[name] = torch.tensor([1.0,0.0,0.0])
        elif choice == 1:
            sample_policy_dict[name] = torch.tensor([0.0,1.0,0.0])
        elif choice == 2:
            sample_policy_dict[name] = torch.tensor([0.0,0.0,1.0])

In [14]:
print(sample_policy_dict)

OrderedDict([('features.models.0.models.0.policy.segment_semantic', tensor([1., 0., 0.])), ('features.models.18.models.0.policy.segment_semantic', tensor([0., 1., 0.])), ('features.models.0.models.0.policy.depth_zbuffer', tensor([1., 0., 0.])), ('features.models.18.models.0.policy.depth_zbuffer', tensor([0., 1., 0.]))])


In [15]:
sample_path = 'checkpoint/CityScapes/'
# app.run(debug=True, use_reloader=False)
sample_state = {'state_dict': sample_policy_dict}
torch.save(sample_state, sample_path + 'sample_policy.model')

### post train from scratch

In [16]:
loss_lambda = {'segment_semantic': 1, 'depth_zbuffer': 1}
trainer.post_train(iters=1, lr=0.001,
                   decay_lr_freq=4000, decay_lr_rate=0.5,
                   loss_lambda=loss_lambda,
                   savePath=checkpoint+'Cityscapes/',
                   reload='sample_policy.model')

Task Policy:
OrderedDict([('features.models.0.models.0.policy.segment_semantic', tensor([1., 0., 0.])), ('features.models.18.models.0.policy.segment_semantic', tensor([0., 1., 0.])), ('features.models.0.models.0.policy.depth_zbuffer', tensor([1., 0., 0.])), ('features.models.18.models.0.policy.depth_zbuffer', tensor([0., 1., 0.]))])


### get the validation results in the paper

In [18]:
for name, param in mtlmodel.named_parameters():
    if 'policy' in name:
        print(param)

Parameter containing:
tensor([1., 0., 0.], requires_grad=True)
Parameter containing:
tensor([1., 0., 0.], requires_grad=True)
Parameter containing:
tensor([0., 1., 0.], requires_grad=True)
Parameter containing:
tensor([0., 1., 0.], requires_grad=True)


In [15]:
for name, param in mtlmodel.named_parameters():
    print(name, param)
mtlmodel.load_state_dict(torch.load('CityScapes.model', map_location=torch.device('cpu')))
trainer.validate('mtl', hard=True) 

heads_dict.segment_semantic.fc1.conv1.weight Parameter containing:
tensor([[[[-2.5540e-03, -6.4725e-03, -2.6625e-05],
          [-8.8473e-03,  8.2442e-03,  7.5032e-04],
          [-2.9728e-04,  3.1306e-03, -4.1111e-03]],

         [[ 7.1913e-03,  2.1458e-03,  3.9305e-03],
          [-2.5746e-03,  1.0129e-03,  6.3021e-03],
          [ 1.6220e-03,  1.7403e-03, -4.2790e-03]],

         [[-9.6010e-06,  7.4598e-03, -4.2133e-03],
          [ 6.7503e-03,  6.2094e-03,  5.0677e-03],
          [ 5.5853e-03,  6.8420e-03,  8.1948e-03]],

         ...,

         [[-6.3459e-03, -8.9415e-03, -2.1001e-03],
          [ 4.6781e-03, -5.4513e-04,  8.4906e-03],
          [ 1.9127e-03,  1.8062e-03,  3.5274e-03]],

         [[ 2.4216e-03, -4.4687e-04,  7.5943e-03],
          [-3.4246e-03, -1.9527e-03,  5.4200e-03],
          [ 3.3485e-03,  1.0436e-03, -1.3098e-03]],

         [[-7.6122e-03, -6.3048e-03,  4.6127e-03],
          [-5.9641e-03, -1.2712e-03, -9.1868e-03],
          [-6.0454e-03,  2.7095e-03, -7.6

RuntimeError: Error(s) in loading state_dict for MobileNetV2:
	Missing key(s) in state_dict: "heads_dict.segment_semantic.fc1.conv1.weight", "heads_dict.segment_semantic.fc1.conv1.bias", "heads_dict.segment_semantic.fc1.conv2.weight", "heads_dict.segment_semantic.fc1.conv2.bias", "heads_dict.segment_semantic.fc1.conv3.weight", "heads_dict.segment_semantic.fc1.conv3.bias", "heads_dict.segment_semantic.fc2.conv1.weight", "heads_dict.segment_semantic.fc2.conv1.bias", "heads_dict.segment_semantic.fc2.conv2.weight", "heads_dict.segment_semantic.fc2.conv2.bias", "heads_dict.segment_semantic.fc2.conv3.weight", "heads_dict.segment_semantic.fc2.conv3.bias", "heads_dict.segment_semantic.fc3.conv1.weight", "heads_dict.segment_semantic.fc3.conv1.bias", "heads_dict.segment_semantic.fc3.conv2.weight", "heads_dict.segment_semantic.fc3.conv2.bias", "heads_dict.segment_semantic.fc3.conv3.weight", "heads_dict.segment_semantic.fc3.conv3.bias", "heads_dict.segment_semantic.fc4.conv1.weight", "heads_dict.s

# Inference

In [7]:
x = torch.from_numpy(np.random.random([32, 3, 3, 3])).float()
# mtlmodel.load_state_dict(torch.load('CityScapes.model', map_location=torch.device('cpu')))
output = mtlmodel(x, task='segment_semantic', hard=True)

tensor([[[[0.0571, 0.2801, 0.5764],
          [0.3677, 0.7520, 0.2789],
          [0.0241, 0.0108, 0.6907]],

         [[0.2849, 0.4900, 0.5635],
          [0.5711, 0.9874, 0.8800],
          [0.3740, 0.7746, 0.2869]],

         [[0.9779, 0.9676, 0.4682],
          [0.1844, 0.2161, 0.5437],
          [0.9895, 0.3826, 0.5238]]],


        [[[0.3233, 0.3036, 0.7744],
          [0.0696, 0.3900, 0.5583],
          [0.7250, 0.3157, 0.7931]],

         [[0.5041, 0.0995, 0.7459],
          [0.8109, 0.8774, 0.0428],
          [0.8579, 0.8018, 0.1505]],

         [[0.0056, 0.9682, 0.2665],
          [0.8756, 0.2590, 0.5863],
          [0.3500, 0.6540, 0.0174]]],


        [[[0.5727, 0.9846, 0.8198],
          [0.4826, 0.5311, 0.4319],
          [0.8454, 0.6797, 0.1137]],

         [[0.4813, 0.6936, 0.8828],
          [0.4113, 0.5065, 0.1531],
          [0.8214, 0.1739, 0.6575]],

         [[0.5113, 0.8164, 0.5099],
          [0.1628, 0.2738, 0.0112],
          [0.8979, 0.9199, 0.4446]]],


    